# 2b. 2-D Riemann Solvers, applied to Blast Waves

## Hydrodynamics
The equations of hydrodynamics are
\begin{align}
\partial_t \rho + \mathbf{\nabla} \cdot [\rho\,\mathbf{v}] & = 0 \\
\partial_t \rho\,\mathbf{v} + \mathbf{\nabla} \cdot [\rho\, \mathbf{v}\otimes\mathbf{v} + P \mathsf{I}] & = 0 \\
\partial_t E + \mathbf{\nabla} \cdot [(E + P)\, \mathbf{v}] & = 0\,,
\end{align}
where $\rho$ is density, $\mathbf{v}$ is velocity, $P$ is pressure, and $E$ is the total energy density. $E = \rho e + {1 \over 2} \rho v^2$, and $e$ is the internal energy per mass. This is complemented by an equation of state $P = (\gamma - 1)\rho e$.

This can also be written in the Lagranian form
\begin{align}
\partial_t \rho & = - \mathbf{v} \cdot \mathbf{\nabla} \rho
- \rho \mathbf{\nabla} \cdot \mathbf{v} \\
\partial_t \mathbf{v} & = - (\mathbf{v} \cdot \mathbf{\nabla}) \mathbf{v}
- \frac{1}{\rho} \mathbf{\nabla} P  \\
\partial_t P &= - \mathbf{v} \cdot \mathbf{\nabla} P
- \gamma P \mathbf{\nabla} \cdot \mathbf{v}\,,
\end{align}
where the advective derivative ($-\mathbf{v} \cdot \mathbf{\nabla}$) is explicit.

Today, we will switch to using 2-D solvers, so we can study for example _blast waves_.

### Standard libaries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from time import time

## Using a Python class to store things
We start by defining a class `hd`, with things we will need for 2-D hydrodynamics.  Notice how we now use Python ___classes___ to make it possible to include also _methods_ (functions, subroutines), bound to the classes. 

In [2]:
class void():
    """ Empty class, used to create ad hoc objects """
    pass

class hd():
    """ Template class for hydrodynamics, with parameters, variables and methods """
    def __init__(self,n=64,gamma=1.4):
        """ Initialization of arrays for conservative variables """
        self.n=n
        self.gamma = gamma
        self.rho   = np.ones((n,n))
        self.Etot  = np.ones((n,n))
        self.Px    = np.zeros((n,n))
        self.Py    = np.zeros((n,n))
        self.coordinates()
    
    def coordinates(self):
        """ Coordinate initialization """
        n = self.n
        self.ds = 2.0/n
        self.dx = self.ds
        self.Lbox = n*self.ds
        self.x = self.ds*(np.arange(n)-n/2+0.5)
        self.y = self.x
        self.r = np.zeros((n,n))
        for i in range(n):
            self.r[i] = (self.y**2 + self.x[i]**2)**0.5

    def velocity(self):
        """ Compute velocity from conservative variables and return in (2,n,n) array """
        return np.array([self.Px/self.rho, self.Py/self.rho])

    def pressure(self):
        """ Compute pressure from conservative variables """
        if (self.gamma==1.0):
            P = self.cs**2 * self.rho
        else:
            Eint = self.Etot - 0.5*(self.Px**2 + self.Py**2)/self.rho 
            P = (self.gamma-1.)*Eint
        return P

    def temperature(self):
        """ Compute the 'temperature', defined as P/rho = kB / mu T """
        return self.pressure() / self.rho
    
    def Courant(self,C=0.5):
        """ Courant condition for HD """
        v = self.velocity()
        speed = np.sqrt(v[0]**2 + v[1]**2)
        cs = np.sqrt(self.gamma*self.pressure()/self.rho)
        return C*self.ds/np.max(cs+speed)

## Riemann Solvers

Riemann solvers are used to compute the fluxes that pass mass, momentum, and energy from cell to cell.  When working in two dimensions, we need to treat the interfaces on all four sides of each cell, with differences between _left_ and _right_, and _bottom_ and _top_ controlling the changes of conserved quantities.

#### __Task 1: Complete the 2-D LLF and HLL solvers (20p)__
Given the description (in the comments below) of how to combine the fluxes, and what you learned in the Tuesday exercise, add the energy equation to the LLF solver, and complete the HLL Riemann solver

In [3]:
# Conservative variables computed from primitve variable
def primitive_to_conservative(q):
    U = void()
    U.D  = q.D
    U.mU = q.D*q.U
    U.mV = q.D*q.V
    # We need an energy U.Etot = ...
    U.Etot=q.P/(q.gamma-1)+1/(2*q.D)*((q.U+q.V)**2)
    return U

# Hydro flux from conservative and primitive variables
def Hydro_Flux(q,U):
    F = void()
    F.D = U.mU
    F.mU = U.mU*q.U + q.P
    F.mV = U.mV*q.U
    # We need an energy flux F.Etot = ...
    F.Etot=(U.Etot+q.P)*q.U
    return F

### LLF fluxes
LLF is the most diffuse Riemann solver. But also the most stable.

In [4]:
def LLF(ql,qr):
    # sound speed for each side of interface (l==left, r==right)
    c_left  = (ql.gamma*ql.P/ql.D)**0.5
    c_right = (qr.gamma*qr.P/qr.D)**0.5
    c_max = np.maximum (c_left, c_right)
    
    # maximum propagation speed anywhere
    cmax = np.maximum (np.abs(ql.U)+c_max, np.abs(qr.U)+c_max)

    # Hydro conservative variable
    Ul = primitive_to_conservative(ql)
    Ur = primitive_to_conservative(qr)

    # Hydro fluxes
    Fl = Hydro_Flux(ql,Ul)
    Fr = Hydro_Flux(qr,Ur)
    
    # LLF flux based on maximum wavespeed.
    # The general form is 
    #   (F_left + F_right - cmax*(U_right - U_left)) / 2
    # where U = (rho, rho*U, rho*V, E_tot) are the conserved variables
    Flux = void()
    Flux.D  = 0.5*(Fl.D  + Fr.D  - cmax*(Ur.D  - Ul.D))
    Flux.mU = 0.5*(Fl.mU + Fr.mU - cmax*(Ur.mU - Ul.mU))
    Flux.mV = 0.5*(Fl.mV + Fr.mV - cmax*(Ur.mV - Ul.mV))
    # Flux.Etot = ...
    Flux.Etot = 0.5*(Fl.Etot + Fr.Etot - cmax*(Ur.Etot - Ul.Etot))
    return Flux

### HLL (Harten, Lax, van Leer) fluxes
This is a bit less diffuse.  The first part is identical to the LFF solver, and only the method to combine the left and right fluxes at the interfaces differ. 

In [5]:
def HLL(ql,qr):
    # sound speed for each side of interface (l==left, r==right)
    c_left  = (ql.gamma*ql.P / ql.D)**0.5
    c_right = (qr.gamma*qr.P / qr.D)**0.5
    c_max = np.maximum(c_left,c_right)
    
    # maximum wave speeds to the left and right (guaranteed to have right sign)
    SL = np.minimum(np.minimum(ql.U,qr.U)-c_max,0) # <= 0.
    SR = np.maximum(np.maximum(ql.U,qr.U)+c_max,0) # >= 0.

    # Hydro conservative variable
    Ul = primitive_to_conservative(ql)
    Ur = primitive_to_conservative(qr)

    # Hydro fluxes
    Fl = Hydro_Flux(ql,Ul)
    Fr = Hydro_Flux(qr,Ur)

    # HLL flux based on wavespeeds. If SL < 0 and SR > 0 then mix state appropriately
    # The general form is "(SR * F_left - SL * F_right + SL * SR *(U_right - U_left)) / (SR - SL)"
    # where U is the state vector of the conserved variables
    Flux = void()
    Flux.D  = (SR*Fl.D  - SL*Fr.D  + SL*SR*(Ur.D  - Ul.D )) / (SR - SL)
    Flux.mU = (SR*Fl.mU - SL*Fr.mU + SL*SR*(Ur.mU - Ul.mU)) / (SR - SL)
    Flux.mV = (SR*Fl.mV - SL*Fr.mV + SL*SR*(Ur.mV - Ul.mV)) / (SR - SL)
    #added this
    
    Flux.Etot =(SR*Fl.Etot - SL*Fr.Etot + SL*SR*(Ur.Etot - Ul.Etot)) / (SR - SL)
#     Flux.D = ...
#     Flux.mU = ...
#     Flux.mV = ...
#     Flux.Etot = ...
    return Flux

### Slope limiter
Monotonized Central (MonCen) slope limiter.  In two dimensions, return the  slopes in both directions, with shape (2,n,n)

In [6]:
def left_slope(f,axis=0):
    return (f-np.roll(f,1,axis))

def MonCen(f):
    """ Monotonized central slope limiter """
    if f.ndim==1:
        ls=left_slope(f)
        rs=np.roll(ls,-1)
        cs=np.zeros(ls.shape)
        w=np.where(ls*rs>0.0)
        cs[w]=2.0*ls[w]*rs[w]/(ls[w]+rs[w])
        return cs
    else:
        shape=np.insert(f.shape,0,f.ndim)
        slopes=np.zeros(shape)
        for i in range(f.ndim):
            ls=left_slope(f,axis=i)
            rs=np.roll(ls,-1,axis=i)
            cs=np.zeros(f.shape)
            w=np.where(ls*rs>0.0)
            cs[w]=2.0*ls[w]*rs[w]/(ls[w]+rs[w])
            slopes[i]=cs
        return slopes

### MUSCL Method for time updates
To simplfy the equations we write the system in vector notation, with a _state vector_ $U$, and a _flux vector_ $F$, and assume we have one dimension. Then
\begin{align}
U &= (\rho, \rho u) \\
F &= (\rho u, \rho u u + P) \\
\partial_t U + \partial_x F & = 0
\end{align}

From last week's lecture slide we have the __master equation__ for solving the system:
\begin{equation}
U(t + \Delta t,x_{i-1/2}) - U(t,x_{i-1/2}) = \frac{\Delta t}{\Delta x} \left[ \tilde F(t + {\Delta t \over 2},x_{i-1}) - \tilde F(t + {\Delta t \over 2},x_{i}) \right]\,,
\end{equation}
where the flux terms are integrals over the time step and the cell faces (that in 1D is just a point, but in 2D a line, and in 3D an area)
\begin{equation}
\tilde F(t + {\Delta t \over 2},x_{i}) = {1 \over \Delta t}
\int^{t + \Delta t}_t \textrm{d} t' F(t', U_I(t',x_i))\,,
\end{equation}
and $U_I$ are the point values of $U$ at the interface.

The MUSCL method improves on Godunov method by approximating the time integral and assuming that we can make a slope interpolation for the values at the interface.
The alogrithm goes as follows
  1. From the state vector $U$ compute the _primitive_ variables q = ($\rho$,$v$,$P$)
  2. Compute the spatial slopes from center of cell to interface. Making the slopes monotonized (TVD) if needed. The slope accross a cell are $\Delta q_x$.
  3. Compute a naive time evolution for the primitve variables at the center of the cell using an Euler step and the Lagranian hydro equations. E.g.
\begin{equation}
\Delta q_t = \Delta t \frac{\textrm{d}q}{\textrm{d}t}
\end{equation}
  4. Approximate $q(t+\Delta/2, x\pm\Delta x/2) = q(t,x) + (\Delta q_t \pm \Delta q_x) / 2$
  5. Shift point of view from cells to interfaces and consider fluid variables on the _left_ and _right_ sides of _interfaces_.
  6. Use an (approximate) Riemann solver to compute the fluxes.

Below is the code corresponding to the algorithm. 

In [7]:
def muscl_2d (u, dt, Slope=MonCen, Riemann_Solver=LLF):
    dtds = dt/u.ds

    # 1) Compute primitive variables (rho, v, P)
    # make sure to copy u.rho to D, otherwise D becomes a "pointer" to u.rho, and updates itself in 5)
    D = np.copy(u.rho)         # D: density
    v = u.velocity()           # v: velocity
    P = u.pressure()           # P: pressure

    # 2) Compute slopes based on centered points
    dD = Slope(D)              # returns (2,n,n)
    dP = Slope(P)              # returns (2,n,n)
    dv0 = Slope(v[0])          # returns (2,n,n)
    dv1 = Slope(v[1])          # returns (2,n,n)
    dv = np.array([dv0,dv1])   # shape = (2,2,n,n)

    # 3) Trace forward to find solution at [t+dt/2, x +- dx/2]
    # Time evolution with source terms
    div = dv[0,0] + dv[1,1]                  # div(v)
    D_t = - v[0]*dD[0] -v[1]*dD[1] - D*div   # dD/dt = -v*grad(D) - D*div(v)
    P_t = - v[0]*dP[0] -v[1]*dP[1] - u.gamma*P*div   # dP/dt = -v*grad(P) - gamma*P*div(v)
    v_t = np.copy(v)                         # shape = (2,n,n)
    v_t[0] = - v[0]*dv[0,0] - v[1]*dv[0,1] - dP[0]/D  # dv/dt = -v*grad(v) - grad(P)/D
    v_t[1] = - v[0]*dv[1,0] - v[1]*dv[1,1] - dP[1]/D
    
    # Loop over X- and Y-direction
    for axis in range(2):
        ql = void(); ql.gamma = u.gamma
        qr = void(); qr.gamma = u.gamma
        
        # Calculates the parallel and perpendicular velocities to the coordinate direction
        # X-axis: iu=0, iv=1, Y-axis: iu=1, iv=0
        iu = axis
        iv = (axis + 1) % 2
        # Parallel (U) and perpendicular (V) velocities
        U = v[iu]; dU = dv[iu]; U_t = v_t[iu]
        V = v[iv]; dV = dv[iv]; V_t = v_t[iv]

        # 4) Spatial interpolation "axis"-direction + time terms

        # left state -- AS SEEN FROM THE INTERFACE
        ql.D = D + 0.5*dD[axis] + 0.5*dtds*D_t
        ql.P = P + 0.5*dP[axis] + 0.5*dtds*P_t
        ql.U = U + 0.5*dU[axis] + 0.5*dtds*U_t
        ql.V = V + 0.5*dV[axis] + 0.5*dtds*V_t

        # right state -- AS SEEN FROM THE INTERFACE
        qr.D = D - 0.5*dD[axis] + 0.5*dtds*D_t
        qr.P = P - 0.5*dP[axis] + 0.5*dtds*P_t
        qr.U = U - 0.5*dU[axis] + 0.5*dtds*U_t
        qr.V = V - 0.5*dV[axis] + 0.5*dtds*V_t

        # 5) Roll down -1 in the "axis" direction so that we collect
        #    left and right state (as seen from the interface) at the same grid index
        qr.D = np.roll(qr.D,-1,axis=axis)
        qr.P = np.roll(qr.P,-1,axis=axis)
        qr.U = np.roll(qr.U,-1,axis=axis)
        qr.V = np.roll(qr.V,-1,axis=axis)

        # 6) Solve for flux based on interface values
        Flux = Riemann_Solver(ql,qr)

        # Update conserved variables with fluxes
        u.rho  -= dtds * (Flux.D - np.roll(Flux.D,1,axis=axis))
        u.Etot -= dtds * (Flux.Etot - np.roll(Flux.Etot,1,axis=axis))
        if axis==0: # X-axis
            u.Px   -= dtds * (Flux.mU - np.roll(Flux.mU,1,axis=axis))
            u.Py   -= dtds * (Flux.mV - np.roll(Flux.mV,1,axis=axis))
        if axis==1: # Y-axis
            u.Px   -= dtds * (Flux.mV - np.roll(Flux.mV,1,axis=axis))
            u.Py   -= dtds * (Flux.mU - np.roll(Flux.mU,1,axis=axis))

    return u

# Blast Wave in 2-D

Releasing a large amount of energy into a background fluid creates an explosion characterized by a strong shock wave, which then expands radially outwards from the point where the energy was released. In astrophysics, this happens, for example, in the case of supernovae.

This is called a Sedov-Taylor blast wave. Sedov and Taylor first solved the problem in the context of atomic bomb explosions, but the solution is also useful in astrophysics. It can be used to test and validate hydrodynamical computer codes since an analytical solution exists. Blast waves are good candidates for adaptive mesh refinement (AMR) technique, that you read about in the book, but they can also be resolved with a uniform grid that we will use in this exercise.

#### __Task 2: Add a profile that defines the blast (10p)__

Complete the blast wave setup with initial profiles for the total energy, of the form

\begin{equation}\tag{2}
f(r) = 1 + A \,\left(\frac{3}{w}\right)^2 e^{-\left(\frac{r}{w\Delta r}\right)^p}
\end{equation}

where $p$ is an exponent of order $2-4$, and $w$ controls the with in cell size ($\Delta r$) units.  In the code, use `e0` for energy, corresponding to $A$ here. The factor $(3/w)^2$ is there to normalize the energy to be the same no matter what is chosen for $w$. It is an approximation, since the normalization will depend on $p$.

In [8]:
class blast_wave(hd):
    """ An extension of the hd() class with initial conditions """
    t = 0.0
    def __init__(u,n=64,gamma=1.4,e0=1e3,d0=1.0,power=2,w=3.,eps=0.01):
        hd.__init__(u,n)
        u.power = power
        u.w = w*u.ds
        u.gamma = gamma
        factor = (3./w)**2
        u.rho  = d0*np.ones((n,n))
        u.Etot = 1.0 + e0*factor*np.exp((-u.r/(w*u.ds))**2)

We want the $y$ coordinate increasing upwards in image plots, so we define a variant of `imshow` (change default `size` according to taste):

In [9]:
def imshow(f,size=10):
    plt.figure(figsize=(size,size))
    plt.imshow(np.transpose(f),origin='lower')
    plt.colorbar();
def title(e0=1e5,d0=1.0,C=0.5,n=128,solver=HLL):
    plt.title('e0={:.0e}   d0={:.0e}   C={:.1f}  n={}  {}'.format(e0,d0,C,n,solver.__name__));

Try it out on the initial condition:

In [10]:
u = blast_wave(n=128,w=20)
imshow(np.log(u.Etot))
imshow(np.log(u.rho))

TypeError: unsupported operand type(s) for +: 'float' and 'ellipsis'

#### __Task 3: Scaling relations (20p)__

Even without the exact analytical solution, one can already test the correctness using the scaling behaviour of the expanding shock.

The radius of an expanding 3-D blast wave in a homogeneous medium scales as

\begin{equation}\tag{1}
R \sim E^{1/5} \rho^{-1/5} t^{2/5}
\end{equation}

We would like to test if we can reproduce the scaling of the shock radius. To do this, run the eperiment, and figure out a way to track the radius as a function of time (search for a suitable maximum as a function of distance from the center). Does it match the expectation, or else how does it scale with time? Produce a plot of the shock radius as a function of time. Check in the output PDFs that the printout actually matches the edge of the shock and that the maximum of the density is not at some other place.

The dimensional analysis assumes a blast wave expanding in a three dimensional medium with a density having dimensions $\rho \sim M/L^3$. But we are running in two dimensions. Derive the scaling law for two dimensions by figuring out the unique combination of time ($t$), density ($\Sigma = M/L^2$), and energy ($M L^2 / S^2$) that gives the appropriate length scale. Does this now match with the shock radius observed in the simulation?

### A small Blast Wave example

This should take about 4 seconds:

In [ ]:
e0 = 1e5
n = 128
C = 0.5
nt = 300
solver = HLL
u = blast_wave(n=n,e0=e0,gamma=1.4,w=3.,power=4,eps=0.1)
start = time()
for it in range(nt):
    dt=u.Courant(C)
    u = muscl_2d(u,dt,Slope=MonCen,Riemann_Solver=solver)
    u.t += dt
used=time()-start
print('{:.1f} sec, {:.2f} microseconds/update'.format(used,1e6*used/(n**2*nt)))
imshow(u.rho)
title(e0=e0,C=C,n=n,solver=solver)
imshow(u.temperature())
title(e0=e0,C=C,n=n,solver=solver)

You may want to use this cheap setup to explore the effect of for example making the width $w$ smaller (to keep things as similar as possible, we added a factor in the peak energy in proportion to $1/w^2$)

### A Blast Wave example, with tendencies for instabilities

Define a new function that runs an experiment with a given solver and resolution and makes a few plots

In [ ]:
def example(n=400,nt=-1,solver=HLL,d0=1.,e0=1e5):
    C = 0.5
    if nt==-1:
        nt = n
    estimate=6.*(n/200.)**3 * nt / n
    if estimate>9.9:
        print("this will take approx {:.0f} seconds".format(estimate))
    u = blast_wave(n=n,e0=e0,gamma=1.4,w=3.0,power=4)
    start = time()
    for it in range(nt):
        dt=u.Courant(C)
        u = muscl_2d(u,dt,Slope=MonCen,Riemann_Solver=solver)
        u.t += dt
    used=time()-start
    print('{:.1f} sec, {:.2f} microseconds/update'.format(used,1e6*used/(n**2*nt)))
    imshow(u.rho)
    title(e0=e0,d0=d0,C=C,n=n,solver=solver)
    imshow(u.temperature())
    title(e0=e0,d0=d0,C=C,n=n,solver=solver)
    return u

Run two quick examples with LLF and HLL solvers. Notice, by plotting e.g. the density, how LLF is more diffusive than HLL.

In [ ]:
u1=example(n=128,solver=LLF)

In [ ]:
u2=example(n=128,solver=HLL)

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(u1.rho[:,u1.n//2],'-o',label='LLF')
plt.plot(u2.rho[:,u2.n//2],'-o',label='HLL')
plt.legend()
plt.xlim(28,45);

To explore the substructure in the blast wave, we need higher resolution. This example should take about 20 seconds to run, and produce a thinner shock front:

In [ ]:
u=example(n=300,solver=HLL)

#### __Task 4: Rayleigh-Taylor instabilities in blast waves (30p)__

A supernova exploding in the interstellar medium does not only have a high pressure, it also contains a significant amount of mass. To simulate this, try adding mass to your experiment, by using a similar profile for density, with `d0` corresponding to $A$.

Add mass by setting `d0` to something similar to but less that `e0`, corresponding to a dense and warm source of the blast. Run the simulation followed by the analysis again. What is the impact of adding mass? How does the expansion change? How does the radius as a function of time change? What about the structure of the shock: Does it look similar to what you saw in the first run? If not, why not?

And: Can you make significant Rayleigh-Taylor instabilities develop, by tuning the parameters (include increasing `n`)?  Note that when you double `n` you also need to double `nit`, to get to the same radius, so the cost (wall clock time) increases with a factor of 8, for each power of 2 (note that it is not necessary to use powers of 2). It may help to add a bit of random noise with amplitude `eps` to e.g. the density profile, to seed the perturbations (what is seeding the perturbations without the noise?).


#### Extra Tasks (and extra points!) for the interested

If you want to have fun with blast waves, you can extend the setup to include several blastwaves, or include areas in your "ISM" that have different density.

  * What happens if your blaswave collide with another one
  * What happens if a blastwave explodes outside a "cloud" (e.g. a dense circular region)
  * What happens if it explodes on the edge, or inside?
  
What do you learn from your experiments about the effect of feedback from Supernovae in the ISM?

### __Absalon turn-in__

Upload the notebook together with a PDF of the notebook, with the coding tasks completed. Please include comments and figures at the end, below this cell. 